# https://realpython.com/docker-continuous-integration/

In this tutorial, you’ll:

- Run a Redis server locally in a Docker container
- Dockerize a Python web application written in Flask
- Build Docker images and push them to the Docker Hub registry
- Orchestrate multi-container applications with Docker Compose
- Replicate a production-like infrastructure anywhere
- Define a continuous integration workflow using GitHub Actions

In [ ]:
!sudo docker run -d --name redis-server redis
!sudo docker network create page-tratracker-
cker-network

!docker network create page-tracker-network
!docker network connect page-tracker-network redis-server
!docker run --rm -it \
             --name redis-client \
             --network page-tracker-network \
             redis redis-cli -h redis-server

In [ ]:
!docker stop redis-server
!docker rm redis-server
!docker run -d --name redis-server -p 6379:6379 redis
!telnet localhost 6379
!docker inspect redis-server

In [4]:
from redis import Redis

redis = Redis()
redis.incr('page_views')
redis.incr('page_views')

4

In [8]:
redis = Redis(host="127.0.0.1", port=6379)
redis.incr("page_views")
redis = Redis.from_url("redis://localhost:6379/")
redis.incr("page_views")

8

In [11]:
!flask --app src.page_tracker.app run

 * Serving Flask app 'src.page_tracker.app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Sep/2024 13:48:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2024 13:48:23] "GET /favicon.ico HTTP/1.1" 404 -
^C


In [12]:
# available to everything on the same network
!flask --app src.page_tracker.app run --host=0.0.0.0 \
                                        --port=8080 \
                                        --debug

 * Serving Flask app 'src.page_tracker.app'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.12.54:8080
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 263-545-886
127.0.0.1 - - [01/Sep/2024 13:49:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2024 13:49:42] "GET /favicon.ico HTTP/1.1" 404 -
192.168.12.54 - - [01/Sep/2024 13:49:44] "GET / HTTP/1.1" 200 -
192.168.12.54 - - [01/Sep/2024 13:49:44] "GET /favicon.ico HTTP/1.1" 404 -
192.168.12.173 - - [01/Sep/2024 13:50:13] "GET / HTTP/1.1" 200 -
192.168.12.173 - - [01/Sep/2024 13:50:13] "GET /favicon.ico HTTP/1.1" 404 -
192.168.12.173 - - [01/Sep/2024 13:50:16] "GET / HTTP/1.1" 200 -
^C


## Perform Static Code Analysis and Security Scanning

In [ ]:
!python -m black src/ --check
!python -m black src/
!python -m isort src/ --check
!python -m isort src/
!python -m flake8 src/
!python -m pylint src/
# !python -m pylint src/ --exit-zero
!python -m bandit -r src/

## Generate requirements.txt

In [ ]:
!poetry export -f requirements.txt --dev --without-hashes --output requirements-dev.txt
!poetry export -f requirements.txt --without-hashes --output requirements.txt

## Docker Images

In [ ]:
!docker build -t page-tracker .
!docker build -f Dockerfile.dev -t page-tracker:dev .
!docker build -f Dockerfile -t page-tracker:prod .

## Git docker images

In [ ]:
!git init
!curl -sL https://www.gitignore.io/api/python,pycharm+all > .gitignore
!git add *
!git commit -m 'initial commit'
!git rev-parse HEAD
!docker build -t page-tracker:$(git rev-parse --short HEAD) .
!docker push pietrotanure/page-tracker:$(git rev-parse --short HEAD)

## Docker run

In [ ]:
!docker run -p 80:5000 --name web-service page-tracker:latest

In [ ]:
!curl http://localhost

In [ ]:
# create container named redis-service from image redis:latest in 
# detached mode (-d) with volume mounted inside /data folder in the container, network
!docker run -d \
             -v redis-volume:/data \
             --network page-tracker-network \
             --name redis-service \
             redis:latest

In [ ]:
#creaate container names web-service from image page-tracker:d4a80d2, with 
# env variable REDIS_URL, with network page-tracker-network, expose container port 5000 on port 80
!docker run -d \
             -p 80:5000 \
             -e REDIS_URL=redis://redis-service:6379 \
             --network page-tracker-network \
             --name web-service \
             page-tracker:d4a80d2

In [ ]:
!curl http://localhost
!curl http://localhost
!curl http://localhost

## Orchestrate containers with Docker Compose

In [ ]:
!docker stop -t 0 web-service redis-service
!docker container rm web-service redis-service
!docker network rm page-tracker-network
!docker volume rm redis-volume

In [ ]:
!docker compose up -d
!docker compose ps

Createion of new port 6379 that is forwarded to host machine

In [ ]:
!docker compose build
!docker compose up --build #build updated .yml file

you can now run your end-to-end tests using pytest installed in a virtual environment on your development machine:

In [ ]:
!python -m pytest web/test/e2e/ \
  --flask-url http://localhost \
  --redis-url redis://localhost:6379

# Create container to run test end-to-end
running the end-to-end test locally against publicly exposed services, you can run it from another container on the same network. You could craft such a container manually. However, recent versions of Docker Compose provide a more elegant solution, which lets you run subsets of services conditionally. You do this by assigning the desired services to custom profiles that you can activate on demand.

In [ ]:
!docker compose --profile testing up -d
!docker compose ps -a
!docker compose logs test-service